# Regression Model Using Keras

Import the required libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

In [4]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape # shape of the datasets

(1030, 9)

Checking the datasets for any missing values by using descibe() method.

In [6]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

As we can see there is no missing values in the datasets. So we are read to build the regression model using keras.

### Splitting datasets into predictors and target 

Here the target variable is the Strength and all columns except strength is the predictors.

In [8]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [9]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [28]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

## A. Build a Baseline model 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

In [24]:
# define regression model
ncols=predictors.shape[1]
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(ncols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and test the network

In [29]:
model=regression_model()

In [30]:
list_of_mean_squared_error=[]
for i in range(50):
    xtrain,xtest,ytrain,ytest=train_test_split(predictors,target,test_size=0.3) # spliting datasets into training(70%) and testing set(30%)
    res=model.fit(xtrain,ytrain, epochs=50,verbose=0,validation_data=(xtest,ytest))# storing the value in variable res
    mean_squared_error=res.history['val_loss'][-1]
    # Adding a value of mean_squared_error in the list 
    list_of_mean_squared_error.append(mean_squared_error)
    print(" Epoch {} : mean_squared_error = {} ".format(i+1,mean_squared_error))
    

 Epoch 1 : mean_squared_error = 376.09999800499975 
 Epoch 2 : mean_squared_error = 76.27648970224325 
 Epoch 3 : mean_squared_error = 78.5992853109119 
 Epoch 4 : mean_squared_error = 66.29280242179205 
 Epoch 5 : mean_squared_error = 63.95407272388248 
 Epoch 6 : mean_squared_error = 60.25543820279316 
 Epoch 7 : mean_squared_error = 55.496926786058545 
 Epoch 8 : mean_squared_error = 43.99102012856493 
 Epoch 9 : mean_squared_error = 45.9592416510227 
 Epoch 10 : mean_squared_error = 54.98985430029218 
 Epoch 11 : mean_squared_error = 46.00450751542274 
 Epoch 12 : mean_squared_error = 49.67146088930395 
 Epoch 13 : mean_squared_error = 51.354820165047755 
 Epoch 14 : mean_squared_error = 48.800590391683734 
 Epoch 15 : mean_squared_error = 55.53482118625085 
 Epoch 16 : mean_squared_error = 48.70208662458994 
 Epoch 17 : mean_squared_error = 48.204252668954794 
 Epoch 18 : mean_squared_error = 42.771659184428096 
 Epoch 19 : mean_squared_error = 49.70988322384535 
 Epoch 20 : mean_

Mean and standard deviation of the mean squared errors 

In [32]:
print("Mean of the mean squared errors = ",np.mean(list_of_mean_squared_error))
print("Standard deviation of the mean squared errors = ",np.std(list_of_mean_squared_error))

Mean of the mean squared errors =  58.25625309397873
Standard deviation of the mean squared errors =  46.079529842090174


## B. Normalize the Data

Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

How does the mean of the mean squared errors compare to that from Step A?

In [19]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [20]:
ncols=predictors_norm.shape[1]
def regression_model2():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(ncols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [21]:
model2=regression_model2()

In [22]:
list_of_mean_squared_error=[]
for i in range(50):
    xtrain,xtest,ytrain,ytest=train_test_split(predictors_norm,target,test_size=0.3) # spliting datasets into training(70%) and testing set(30%)
    res=model.fit(xtrain,ytrain, epochs=50,verbose=0,validation_data=(xtest,ytest))# storing the value in variable res
    mean_squared_error=res.history['val_loss'][-1]
    # Adding a value of mean_squared_error in the list 
    list_of_mean_squared_error.append(mean_squared_error)
    print(" Epoch {} : mean_squared_error = {} ".format(i+1,mean_squared_error))

 Epoch 1 : mean_squared_error = 198.04808824656465 
 Epoch 2 : mean_squared_error = 137.1582934925857 
 Epoch 3 : mean_squared_error = 107.88525279517313 
 Epoch 4 : mean_squared_error = 66.464994782383 
 Epoch 5 : mean_squared_error = 59.08745892225346 
 Epoch 6 : mean_squared_error = 56.311338183949296 
 Epoch 7 : mean_squared_error = 49.67347796295067 
 Epoch 8 : mean_squared_error = 43.466706001257045 
 Epoch 9 : mean_squared_error = 37.5378743267368 
 Epoch 10 : mean_squared_error = 39.80992222758173 
 Epoch 11 : mean_squared_error = 38.2298899777113 
 Epoch 12 : mean_squared_error = 38.28047851760025 
 Epoch 13 : mean_squared_error = 39.67984512477245 
 Epoch 14 : mean_squared_error = 40.96007662529313 
 Epoch 15 : mean_squared_error = 34.79046568515617 
 Epoch 16 : mean_squared_error = 40.5866910446809 
 Epoch 17 : mean_squared_error = 39.44450479649417 
 Epoch 18 : mean_squared_error = 36.91650957274206 
 Epoch 19 : mean_squared_error = 41.77231502841592 
 Epoch 20 : mean_squar

Mean and standard deviation of the mean squared errors

In [23]:
print("Mean of the mean squared errors = ",np.mean(list_of_mean_squared_error))
print("Standard deviation of the mean squared errors = ",np.std(list_of_mean_squared_error))

Mean of the mean squared errors =  46.17599665830051
Standard deviation of the mean squared errors =  27.980187285423995


#### The mean and the standard deviation of the mean squared errors in case A is more than in case B. But the difference is tiny. And in my opinion it's not a very good idea to compare result of two poor neural networks with one hidden layer only. Data normalization does not help a lot. Error is huge for both cases: A and B.

## C. Increase the number of epochs

Repeat Part B but use 100 epochs this time for training.

How does the mean of the mean squared errors compare to that from Step B?



In [34]:
def regression_model3():
    model3 = Sequential()
    model3.add(Dense(10, activation='relu', input_shape=(ncols,)))
    model3.add(Dense(1))
    
    model3.compile(optimizer='adam', loss='mean_squared_error')
    return model3

model3 = regression_model3()

In [35]:
list_of_mean_squared_error=[]
for i in range(100):
    xtrain,xtest,ytrain,ytest=train_test_split(predictors_norm,target,test_size=0.3) # spliting datasets into training(70%) and testing set(30%)
    res=model.fit(xtrain,ytrain, epochs=100,verbose=0,validation_data=(xtest,ytest))# storing the value in variable res
    mean_squared_error=res.history['val_loss'][-1]
    # Adding a value of mean_squared_error in the list 
    list_of_mean_squared_error.append(mean_squared_error)
    print(" Epoch {} : mean_squared_error = {} ".format(i+1,mean_squared_error))

 Epoch 1 : mean_squared_error = 151.074944184436 
 Epoch 2 : mean_squared_error = 105.25281734528279 
 Epoch 3 : mean_squared_error = 96.882188938968 
 Epoch 4 : mean_squared_error = 71.221511124793 
 Epoch 5 : mean_squared_error = 54.0272725670083 
 Epoch 6 : mean_squared_error = 51.73688102154284 
 Epoch 7 : mean_squared_error = 39.77064759831598 
 Epoch 8 : mean_squared_error = 40.64334010151983 
 Epoch 9 : mean_squared_error = 37.71705388559879 
 Epoch 10 : mean_squared_error = 32.84514150033105 
 Epoch 11 : mean_squared_error = 45.05835991152668 
 Epoch 12 : mean_squared_error = 34.480482601425024 
 Epoch 13 : mean_squared_error = 41.22263474448985 
 Epoch 14 : mean_squared_error = 31.882426635347137 
 Epoch 15 : mean_squared_error = 38.31013603580808 
 Epoch 16 : mean_squared_error = 37.519990063022256 
 Epoch 17 : mean_squared_error = 37.08647084158987 
 Epoch 18 : mean_squared_error = 33.57780324448274 
 Epoch 19 : mean_squared_error = 36.855074799176556 
 Epoch 20 : mean_squar

In [36]:
print('The mean of the mean squared errors = ',np.mean(list_of_mean_squared_error))
print('The standard deviation of the mean squared errors = ',np.std(list_of_mean_squared_error))

The mean of the mean squared errors =  37.90472390039068
The standard deviation of the mean squared errors =  15.61232939777669


### The mean and the standard deviation of the mean squared errors in case C is less than in case B. But in both cases error is huge. In my opinion it's not a very good idea to compare result of two poor neural networks with one hidden layer only. Number of epoch does not help.

## D. Increase the number of hidden layers

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

How does the mean of the mean squared errors compare to that from Step B?

In [37]:
def regression_model4():
    model4 = Sequential()
    model4.add(Dense(10, activation='relu', input_shape=(ncols,)))
    model4.add(Dense(10, activation='relu'))
    model4.add(Dense(10, activation='relu'))
    model4.add(Dense(1))
    
    model4.compile(optimizer='adam', loss='mean_squared_error')
    return model4

In [38]:
model4 = regression_model4()

In [39]:
list_of_mean_squared_error=[]
for i in range(50):
    xtrain,xtest,ytrain,ytest=train_test_split(predictors_norm,target,test_size=0.3) # spliting datasets into training(70%) and testing set(30%)
    res=model.fit(xtrain,ytrain, epochs=50,verbose=0,validation_data=(xtest,ytest))# storing the value in variable res
    mean_squared_error=res.history['val_loss'][-1]
    # Adding a value of mean_squared_error in the list 
    list_of_mean_squared_error.append(mean_squared_error)
    print(" Epoch {} : mean_squared_error = {} ".format(i+1,mean_squared_error))

 Epoch 1 : mean_squared_error = 30.95370004400852 
 Epoch 2 : mean_squared_error = 31.878442400095917 
 Epoch 3 : mean_squared_error = 35.13899706868292 
 Epoch 4 : mean_squared_error = 35.35643921392249 
 Epoch 5 : mean_squared_error = 31.69708287137226 
 Epoch 6 : mean_squared_error = 31.332599726309667 
 Epoch 7 : mean_squared_error = 33.65150364008536 
 Epoch 8 : mean_squared_error = 32.190050813372466 
 Epoch 9 : mean_squared_error = 34.30197187997762 
 Epoch 10 : mean_squared_error = 25.789145448061255 
 Epoch 11 : mean_squared_error = 34.687626773871266 
 Epoch 12 : mean_squared_error = 30.841224423504183 
 Epoch 13 : mean_squared_error = 32.77521421068309 
 Epoch 14 : mean_squared_error = 32.18450870946001 
 Epoch 15 : mean_squared_error = 34.51255676121388 
 Epoch 16 : mean_squared_error = 31.6223039225853 
 Epoch 17 : mean_squared_error = 30.762098898779612 
 Epoch 18 : mean_squared_error = 30.927997638492524 
 Epoch 19 : mean_squared_error = 34.61592000819333 
 Epoch 20 : me

In [40]:
print('The mean of the mean squared errors = ',np.mean(list_of_mean_squared_error))
print('The standard deviation of the mean squared errors = ',np.std(list_of_mean_squared_error))

The mean of the mean squared errors =  31.7572090613109
The standard deviation of the mean squared errors =  2.485823597257911


#### The mean and the standard deviation of the mean squared errors in case D is less than in case A, B and C. And it's the only case where error is not very big. It means additional layers in neural network are more important than other things. Also it proves the comparison between poor neural network with one hidden layer in previous cases is a bad idea. Result can be unpredictable.